In [5]:
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.datasets.mnist import load_data
import numpy as np

# Note: I based my code largely on the tutorial that is available here:
# https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
# and then adapted it to the specific assignment, including modifying it to create a separate GAN for each digit,
# changing the hyperparameters, and adding new plotting functionality. I made sure to read through all the 
# explanatory text while walking through the tutorial to make sure I knew what each part was doing, 
# and why it was structured the way it was.


def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# train the discriminator model
def train_discriminator(model, dataset, n_iter=100, batch_size=256):
    half_batch = int(batch_size / 2)
    # manually enumerate epochs
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator on real samples
        _, real_acc = model.train_on_batch(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(half_batch)
        # update discriminator on fake samples
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        # summarize performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [6]:
# this function takes a random subset of the training data and creates a dataest with "true" (1) labels
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

#this function generates random images and creates a dataet of size n_samples with "fake" (0) labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

# load and prepare mnist training images
def load_real_samples(digit):
    # load mnist dataset
    (trainX, trainY), (_, _) = load_data()
    #print("head of train_X")
    #print(str(trainX[:5]))
    train_ind = np.where(trainY==digit)
    trainX = trainX[train_ind]
    # expand to 3d, e.g. add channels dimension
    X = np.expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [7]:
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model


In [8]:
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def train_gan(gan_model, latent_dim, n_epochs=100, batch_size=256):
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, batch_size)
        # create inverted labels for the fake samples
        y_gan = np.ones((batch_size, 1))
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)

In [65]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, digit,n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save the generator model tile file
    filename = str(digit)+'_ld'+str(latent_dim)+'_generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)
    

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, digit, n_epochs=10, batch_size=256,):
    bat_per_epo = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    # manually enumerate epochs
    print("training GAN model for digit: " + str(digit))
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, batch_size)
            # create inverted labels for the fake samples
            y_gan = np.ones((batch_size, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 5 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim,digit)
            


In [66]:
for i in [3,4,5,6,7,8,9]:
    # size of the latent space
    latent_dim = 5
    # create the discriminator
    d_model = define_discriminator()
    # create the generator
    g_model = define_generator(latent_dim)
    # create the gan
    gan_model = define_gan(g_model, d_model)
    # load image data
    dataset = load_real_samples(i)
    # train model
    train(g_model, d_model, gan_model, dataset, latent_dim,i)

training GAN model for digit: 3
>1, 1/23, d=0.682, g=0.736
>1, 2/23, d=0.675, g=0.753
>1, 3/23, d=0.667, g=0.767
>1, 4/23, d=0.659, g=0.785
>1, 5/23, d=0.648, g=0.805
>1, 6/23, d=0.643, g=0.816
>1, 7/23, d=0.634, g=0.830
>1, 8/23, d=0.623, g=0.855
>1, 9/23, d=0.616, g=0.870
>1, 10/23, d=0.612, g=0.883
>1, 11/23, d=0.600, g=0.897
>1, 12/23, d=0.601, g=0.914
>1, 13/23, d=0.592, g=0.912
>1, 14/23, d=0.590, g=0.913
>1, 15/23, d=0.593, g=0.895
>1, 16/23, d=0.591, g=0.870
>1, 17/23, d=0.597, g=0.842
>1, 18/23, d=0.601, g=0.817
>1, 19/23, d=0.607, g=0.788
>1, 20/23, d=0.604, g=0.769
>1, 21/23, d=0.609, g=0.750
>1, 22/23, d=0.600, g=0.737
>1, 23/23, d=0.593, g=0.730
>2, 1/23, d=0.589, g=0.723
>2, 2/23, d=0.585, g=0.718
>2, 3/23, d=0.580, g=0.717
>2, 4/23, d=0.574, g=0.713
>2, 5/23, d=0.568, g=0.710
>2, 6/23, d=0.554, g=0.710
>2, 7/23, d=0.548, g=0.708
>2, 8/23, d=0.543, g=0.707
>2, 9/23, d=0.526, g=0.705
>2, 10/23, d=0.522, g=0.705
>2, 11/23, d=0.514, g=0.704
>2, 12/23, d=0.501, g=0.704
>2, 13

>3, 7/22, d=0.564, g=0.535
>3, 8/22, d=0.592, g=0.528
>3, 9/22, d=0.599, g=0.514
>3, 10/22, d=0.596, g=0.538
>3, 11/22, d=0.596, g=0.563
>3, 12/22, d=0.576, g=0.572
>3, 13/22, d=0.568, g=0.581
>3, 14/22, d=0.579, g=0.587
>3, 15/22, d=0.588, g=0.575
>3, 16/22, d=0.593, g=0.579
>3, 17/22, d=0.594, g=0.573
>3, 18/22, d=0.595, g=0.579
>3, 19/22, d=0.610, g=0.580
>3, 20/22, d=0.599, g=0.578
>3, 21/22, d=0.619, g=0.589
>3, 22/22, d=0.611, g=0.600
>4, 1/22, d=0.609, g=0.606
>4, 2/22, d=0.612, g=0.613
>4, 3/22, d=0.622, g=0.616
>4, 4/22, d=0.614, g=0.626
>4, 5/22, d=0.619, g=0.629
>4, 6/22, d=0.616, g=0.638
>4, 7/22, d=0.608, g=0.645
>4, 8/22, d=0.620, g=0.653
>4, 9/22, d=0.617, g=0.654
>4, 10/22, d=0.621, g=0.670
>4, 11/22, d=0.623, g=0.676
>4, 12/22, d=0.620, g=0.684
>4, 13/22, d=0.617, g=0.689
>4, 14/22, d=0.623, g=0.700
>4, 15/22, d=0.625, g=0.709
>4, 16/22, d=0.626, g=0.718
>4, 17/22, d=0.619, g=0.728
>4, 18/22, d=0.618, g=0.727
>4, 19/22, d=0.621, g=0.736
>4, 20/22, d=0.625, g=0.741
>4, 

>6, 1/21, d=0.731, g=0.791
>6, 2/21, d=0.701, g=0.766
>6, 3/21, d=0.725, g=0.785
>6, 4/21, d=0.748, g=0.766
>6, 5/21, d=0.718, g=0.758
>6, 6/21, d=0.710, g=0.757
>6, 7/21, d=0.707, g=0.777
>6, 8/21, d=0.683, g=0.746
>6, 9/21, d=0.679, g=0.772
>6, 10/21, d=0.675, g=0.764
>6, 11/21, d=0.660, g=0.774
>6, 12/21, d=0.650, g=0.771
>6, 13/21, d=0.634, g=0.800
>6, 14/21, d=0.637, g=0.797
>6, 15/21, d=0.640, g=0.795
>6, 16/21, d=0.624, g=0.789
>6, 17/21, d=0.624, g=0.793
>6, 18/21, d=0.612, g=0.786
>6, 19/21, d=0.605, g=0.790
>6, 20/21, d=0.615, g=0.796
>6, 21/21, d=0.603, g=0.800
>7, 1/21, d=0.602, g=0.781
>7, 2/21, d=0.593, g=0.790
>7, 3/21, d=0.614, g=0.778
>7, 4/21, d=0.611, g=0.776
>7, 5/21, d=0.634, g=0.763
>7, 6/21, d=0.609, g=0.753
>7, 7/21, d=0.625, g=0.783
>7, 8/21, d=0.643, g=0.783
>7, 9/21, d=0.649, g=0.796
>7, 10/21, d=0.623, g=0.771
>7, 11/21, d=0.629, g=0.782
>7, 12/21, d=0.640, g=0.756
>7, 13/21, d=0.630, g=0.739
>7, 14/21, d=0.660, g=0.739
>7, 15/21, d=0.633, g=0.727
>7, 16/21,

>8, 16/23, d=0.639, g=0.760
>8, 17/23, d=0.642, g=0.773
>8, 18/23, d=0.627, g=0.769
>8, 19/23, d=0.614, g=0.782
>8, 20/23, d=0.608, g=0.797
>8, 21/23, d=0.624, g=0.790
>8, 22/23, d=0.630, g=0.809
>8, 23/23, d=0.616, g=0.806
>9, 1/23, d=0.622, g=0.825
>9, 2/23, d=0.609, g=0.855
>9, 3/23, d=0.602, g=0.854
>9, 4/23, d=0.616, g=0.841
>9, 5/23, d=0.620, g=0.837
>9, 6/23, d=0.623, g=0.827
>9, 7/23, d=0.650, g=0.854
>9, 8/23, d=0.636, g=0.822
>9, 9/23, d=0.640, g=0.837
>9, 10/23, d=0.644, g=0.820
>9, 11/23, d=0.652, g=0.811
>9, 12/23, d=0.644, g=0.805
>9, 13/23, d=0.662, g=0.788
>9, 14/23, d=0.691, g=0.783
>9, 15/23, d=0.666, g=0.738
>9, 16/23, d=0.684, g=0.745
>9, 17/23, d=0.684, g=0.727
>9, 18/23, d=0.687, g=0.730
>9, 19/23, d=0.687, g=0.748
>9, 20/23, d=0.684, g=0.737
>9, 21/23, d=0.669, g=0.737
>9, 22/23, d=0.668, g=0.722
>9, 23/23, d=0.705, g=0.715
>10, 1/23, d=0.690, g=0.689
>10, 2/23, d=0.714, g=0.674
>10, 3/23, d=0.699, g=0.675
>10, 4/23, d=0.688, g=0.679
>10, 5/23, d=0.705, g=0.682
>

>10, 11/24, d=0.688, g=0.745
>10, 12/24, d=0.686, g=0.729
>10, 13/24, d=0.703, g=0.727
>10, 14/24, d=0.691, g=0.733
>10, 15/24, d=0.710, g=0.727
>10, 16/24, d=0.721, g=0.731
>10, 17/24, d=0.705, g=0.716
>10, 18/24, d=0.704, g=0.726
>10, 19/24, d=0.699, g=0.729
>10, 20/24, d=0.710, g=0.737
>10, 21/24, d=0.697, g=0.719
>10, 22/24, d=0.728, g=0.714
>10, 23/24, d=0.726, g=0.721
>10, 24/24, d=0.704, g=0.715
>Accuracy real: 19%, fake: 66%
training GAN model for digit: 8
>1, 1/22, d=0.685, g=0.725
>1, 2/22, d=0.673, g=0.743
>1, 3/22, d=0.667, g=0.751
>1, 4/22, d=0.660, g=0.767
>1, 5/22, d=0.648, g=0.788
>1, 6/22, d=0.642, g=0.795
>1, 7/22, d=0.633, g=0.819
>1, 8/22, d=0.626, g=0.832
>1, 9/22, d=0.616, g=0.851
>1, 10/22, d=0.600, g=0.875
>1, 11/22, d=0.599, g=0.886
>1, 12/22, d=0.591, g=0.900
>1, 13/22, d=0.583, g=0.908
>1, 14/22, d=0.581, g=0.909
>1, 15/22, d=0.576, g=0.902
>1, 16/22, d=0.572, g=0.882
>1, 17/22, d=0.575, g=0.856
>1, 18/22, d=0.577, g=0.825
>1, 19/22, d=0.581, g=0.795
>1, 20/2

>2, 16/23, d=0.473, g=0.720
>2, 17/23, d=0.472, g=0.720
>2, 18/23, d=0.463, g=0.720
>2, 19/23, d=0.450, g=0.721
>2, 20/23, d=0.449, g=0.720
>2, 21/23, d=0.444, g=0.714
>2, 22/23, d=0.442, g=0.708
>2, 23/23, d=0.436, g=0.702
>3, 1/23, d=0.444, g=0.692
>3, 2/23, d=0.448, g=0.671
>3, 3/23, d=0.455, g=0.670
>3, 4/23, d=0.458, g=0.674
>3, 5/23, d=0.479, g=0.649
>3, 6/23, d=0.468, g=0.622
>3, 7/23, d=0.493, g=0.599
>3, 8/23, d=0.505, g=0.593
>3, 9/23, d=0.538, g=0.604
>3, 10/23, d=0.518, g=0.593
>3, 11/23, d=0.550, g=0.612
>3, 12/23, d=0.549, g=0.623
>3, 13/23, d=0.535, g=0.644
>3, 14/23, d=0.500, g=0.632
>3, 15/23, d=0.529, g=0.656
>3, 16/23, d=0.534, g=0.656
>3, 17/23, d=0.530, g=0.690
>3, 18/23, d=0.547, g=0.682
>3, 19/23, d=0.538, g=0.710
>3, 20/23, d=0.553, g=0.708
>3, 21/23, d=0.558, g=0.739
>3, 22/23, d=0.564, g=0.729
>3, 23/23, d=0.549, g=0.777
>4, 1/23, d=0.556, g=0.772
>4, 2/23, d=0.560, g=0.798
>4, 3/23, d=0.556, g=0.807
>4, 4/23, d=0.555, g=0.812
>4, 5/23, d=0.547, g=0.824
>4, 6/

the code below uses the model that was trained on 10 epochs, using 5 latent dimensions. I found 5 latent dimensions to be better than 10 or 100, which was recommended by the tutorial when using a composite model for all 10 digits. When too many latent dimensions were used, the discriminator accuracy would crash, and the generator accuracy would spike, after a certain number of epochs. The pictures it generated ultimately created a checkerboard pattern. 

In [70]:
from keras.models import load_model
from matplotlib import pyplot
def split(word):
    return [char for char in word]

def generate_png(digits_string):
    digit_list = split(digits_string)
    n = len(digit_list)
    print("n: " + str(n))
    count =0
    for digit in digit_list:
        count=count+1
        model = load_model('/Users/paullintilhac/engs108-hw/'+digit+'_ld5_generator_model_010.h5', compile=False)
        latent_points = generate_latent_points(5, 1)
        X = model.predict(latent_points)
        pyplot.subplot(n, 1, count)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[0, :, :, 0], cmap='gray_r')
        pyplot.tight_layout()

        pyplot.show()

generate_png("0123456789")

n: 10
